In [23]:
# Importing packages
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)
import sqlite3

In [2]:
# Setting the path to the dataset
path_22 = 'D:/tableau/spend analysis/data/2022_transaction_download.csv'
path_23 = 'D:/tableau/spend analysis/data/2023_ytd_transaction_download.csv'

In [3]:
# Reading both datasets
df22 = pd.read_csv(path_22)
df23 = pd.read_csv(path_23)

In [4]:
# Merging both datasets
merged = pd.concat([df22, df23], axis=0)

In [5]:
# Taking a look at the dataset
merged.head()

,Transaction Date,Posted Date,Card No.,Description,Category,Debit,Credit
0,2022-12-30,2022-12-31,5296,DOMINO'S 5593,Dining,40.80,NaN
1,2022-12-29,2022-12-29,5296,ELECTRONIC PAYMENT,Payment/Credit,NaN,39.17
2,2022-12-28,2022-12-29,5296,HARRIS TEETER #0205,Merchandise,39.17,NaN
3,2022-12-28,2022-12-28,5296,ELECTRONIC PAYMENT,Payment/Credit,NaN,248.53
4,2022-12-23,2022-12-24,5296,HARRIS TEETER #0205,Merchandise,212.18,NaN


In [6]:
# Converting both dates to datetime format
merged.loc[:, 'Transaction Date'] = pd.to_datetime(merged['Transaction Date'])
merged.loc[:, 'Posted Date'] = pd.to_datetime(merged['Posted Date'])

C:\Users\abhij\AppData\Local\Temp\ipykernel_16568\2696844790.py:2: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  merged.loc[:, 'Transaction Date'] = pd.to_datetime(merged['Transaction Date'])
C:\Users\abhij\AppData\Local\Temp\ipykernel_16568\2696844790.py:3: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  merged.loc[:, 'Posted Date'] = pd.to_datetime(merged['Posted Date'])


In [7]:
# Changing the datatypes of remaining columns
#merged.loc[:, 'Card No.'] = merged['Card No.'].astype('int32')
#merged.loc[:, 'Category'] = merged['Card No.'].astype('category')
#merged.loc[:, 'Debit'] = merged['Debit'].astype('float32')
#merged.loc[:, 'Credit'] = merged['Credit'].astype('float32')

In [8]:
# Sorting the dataset by Transaction Date
merged.sort_values(by='Transaction Date', ascending=True).head()

,Transaction Date,Posted Date,Card No.,Description,Category,Debit,Credit
261,2021-12-31,2022-01-03,5296,PATEL BROS,Merchandise,237.82,NaN
262,2021-12-31,2022-01-03,5296,EXXONMOBIL 48118087,Gas/Automotive,22.59,NaN
260,2022-01-02,2022-01-03,5296,CVS/PHARMACY #07089,Health Care,11.00,NaN
258,2022-01-04,2022-01-04,5296,ELECTRONIC PAYMENT,Payment/Credit,NaN,33.59
259,2022-01-04,2022-01-04,5296,ELECTRONIC PAYMENT,Payment/Credit,NaN,357.82


In [9]:
# Changing the category for Grocery stores to "Grocery"
for name in ['HARRIS TEETER #0205', 'PATEL BROS', 'THE FRESH MARKET 219']:
    merged.loc[merged.loc[:, 'Description'] == name, 'Category'] = 'Grocery'

In [10]:
# Combining Other with Other Services
merged.loc[merged.loc[:, 'Description'] == 'UWAA', 'Category'] = 'Other Services'

In [16]:
# Creating day as a feature
merged.loc[:, 'Day'] = merged['Transaction Date'].dt.day_name()

In [18]:
# Creating month as a feature
merged.loc[:, 'Month'] = merged['Transaction Date'].dt.month_name()

In [20]:
# Creating year as a feature
merged.loc[:, 'Year'] = merged['Transaction Date'].dt.year

In [24]:
# Creating a feature for quarter
condList = [
    (merged['Month'].isin(['January', 'February', 'March'])),
    (merged['Month'].isin(['April', 'May', 'June'])),
    (merged['Month'].isin(['July', 'August', 'September'])),
    (merged['Month'].isin(['October', 'November', 'December'])),
]

choiceList = ['Q1', 'Q2', 'Q3', 'Q4']

merged.loc[:, 'Quarter'] = np.select(condList, choiceList)

In [26]:
# Creating a SQLite connector
#conn = sqlite3.connect('transactions.db')
#merged.to_sql('lnd_spend', conn, if_exists='append', index=False)

335

In [28]:
toSave = merged.loc[merged.loc[:, 'Category'] != 'Payment/Credit', :]

In [31]:
save = toSave[['Transaction Date', 'Description', 'Category', 'Debit', 'Day', 'Month', 'Year', 'Quarter']]

In [32]:
save.to_csv('spend_analysis.csv', index=False)